In [ ]:
# UNUSED AT THIS MOMENT
def rfc9325_recommendation(cipher, tls_version, public_key_size) -> str:
    # https://datatracker.ietf.org/doc/html/rfc9325#name-general-guidelines
    # about cipher suites:
    # https://utcc.utoronto.ca/~cks/space/blog/tech/SSLCipherNames

    # "SHALL" and "MUST"
    cipher_params = cipher.split("_")
    veredict = "Y"
    must_not = ["NULL", "RC4"]
    if any(item in must_not for item in cipher_params):
        veredict = "N"

    for param in cipher_params:
        try:
            key_size = int(param.rstrip("L"))
            if key_size < 128:  # should not; 112 is must not
                veredict = "N"
        except:
            pass

    if "_DES40_" in cipher:
        veredict = "N"

    # the "SHOULD NOT" also taken into account
    should_not = ["TLS_RSA_WITH_",  # e.g. TLS_RSA_WITH_AES_128_CBC_SHA
                  "TLS_DH_", 
                  "TLS_ECDH_"  # https://doi.org/10.1007/978-3-319-24174-6_21
                 ]
    if any(i in cipher for i in should_not):
        veredict = "N"

    if tls_version == "TLSv1.2": 
        recommended_v2 = ["TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256",
                          "TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384",
                          "TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256",
                          "TLS_ECDHE_ECDSA_WITH_AES_256_GCM_SHA384"
                         ]
        if cipher in recommended_v2:
            veredict = "Y"

        if "TLS_DHE_" in cipher:
            veredict = "N"

    # _CBC_ should not be used unless encrypt-then-mac is successfully negotiated
    # encrypt_then_mac extension in client hello with extension_type=0x16 and extension_data=<empty>
    # https://datatracker.ietf.org/doc/html/rfc7366

    # use client hello to detect ECDSA that use NIST curve P-256 and X25519 rfc4492#section-5.1
    # secp256r1 and x25519

    # https://www.rfc-editor.org/rfc/rfc8446#section-9.1
    recommended_v3 = ["TLS_AES_128_GCM_SHA256",
                      "TLS_AES_256_GCM_SHA384",
                      "TLS_CHACHA20_POLY1305_SHA256"
                     ]
    if tls_version == "TLSv1.3":
        if cipher in recommended_v3:
            veredict = "Y"

    # TODO key_exchange_size has to be retrieved from the ConnectionState in the Goscanner
    #if "DHE" in cipher and (key_exchange_size is None or key_exchange_size < 2048):
    #    veredict = "N"
    #if "ECDH" in cipher and (key_exchange_size is None or key_exchange_size < 224):
    #    veredict = "N"

    if "TLS_RSA_WITH_" not in cipher and "RSA" in cipher:
        hash_algo = cipher.split("_")[-1]
        recommended_hash = ["SHA256", "SHA384", "SHA512"]
        not_recommended_hash = ["SHA1", "SHA", "MD5"]
        if hash_algo in recommended_hash:
            veredict = "Y"
        elif hash_algo in not_recommended_hash:
            veredict = "N"

        if public_key_size is None or public_key_size < 2048:
            veredict = "N"

    # if "truncated_hmac": veredict = "N"  # in the extended client hello
    return veredict


is_rfc9325_recommended_udf = psf.udf(rfc9325_recommendation, StringType())